# CAR PRICE ESTIMATION MODEL (EDA - FILLING NULL VALUES)

## IMPORTING LIBRARIES

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import warnings;
warnings.filterwarnings("ignore")
import re
#pd.set_option("display.max_columns",None)
#pd.set_option("display.max_rows",None)

## IMPORTING THE DATAFRAME

In [2]:
auto = pd.read_csv('auto_scout_cleaned.csv')
df = auto.copy()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15919 non-null  int64  
 1   make_model             15919 non-null  object 
 2   price                  15919 non-null  float64
 3   vat                    11406 non-null  object 
 4   km                     14895 non-null  float64
 5   registration           14322 non-null  float64
 6   prev_owner             9254 non-null   float64
 7   hp                     15831 non-null  float64
 8   Type                   15917 non-null  object 
 9   Next Inspection        3535 non-null   float64
 10  Inspection new         15919 non-null  object 
 11  Warranty(months)       4853 non-null   float64
 12  Body Color             15322 non-null  object 
 13  Paint Type             15919 non-null  object 
 14  Body Color Original    15904 non-null  object 
 15  Bo

## FILLING NULL VALUES

### Column 0: Unnamed: 0 

In [4]:
df.drop(columns = ['Unnamed: 0'], inplace=True)

### Column 3: vat

In [5]:
df.vat.value_counts(dropna=False)

VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64

In [6]:
df['vat'] = df['vat'].fillna('VAT non-deductible')

In [7]:
df["vat"].value_counts(dropna=False)

VAT deductible        10980
VAT non-deductible     4513
Price negotiable        426
Name: vat, dtype: int64

### Column 15: Body

In [8]:
df.groupby('make_model')['Body'].value_counts(dropna=False)

make_model      Body         
Audi A1         Sedans           1538
                Compact          1039
                Station wagon      21
                Other              13
                Coupe               2
                Van                 1
Audi A2         Off-Road            1
Audi A3         Sedans           2598
                Station wagon     282
                Compact           182
                Other              16
                Convertible         8
                NaN                 7
                Coupe               4
Opel Astra      Station wagon    1211
                Sedans           1053
                Compact           185
                Other              67
                NaN                 7
                Coupe               2
                Off-Road            1
Opel Corsa      Compact          1230
                Sedans            875
                Other              87
                Coupe              13
                Tran

In [9]:
#Fill the column according to make_model column
df["Body"].fillna(df.groupby('make_model')['Body'].transform(lambda x: x.mode()[0]), inplace=True)

### Column 8: type

In [12]:
#Fill Null values as 'Used'
df['Type'] = df['Type'].fillna('Used')

### Column 5: registration

In [10]:
df.registration.value_counts(dropna=False)

2018.0    4522
2016.0    3674
2017.0    3273
2019.0    2853
NaN       1597
Name: registration, dtype: int64

In [13]:
df['registration']=df['registration'].fillna(df.groupby('Type')['registration'].transform(lambda x:x.mode()[0]))

In [14]:
df.registration.value_counts(dropna=False)

2018.0    4525
2019.0    4411
2016.0    3710
2017.0    3273
Name: registration, dtype: int64

### Column 7: hp

In [18]:
mode_hp = df[~(df['Displacement'].isnull())]\
              .groupby(['make_model','Body','Displacement'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [19]:
df.loc[~(df['Displacement'].isnull()),"hp"] = df.loc[~(df['Displacement'].isnull()),"hp"].fillna(mode_hp)

In [21]:
mode_hp1 = df[~(df['Cylinders'].isnull())]\
              .groupby(['make_model','Body','Cylinders'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [22]:
df.loc[~(df['Cylinders'].isnull()),"hp"] = df.loc[~(df['Cylinders'].isnull()),"hp"].fillna(mode_hp1)

In [23]:
mode_hp2 = df.groupby(['make_model','Body'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [24]:
df['hp'] = df['hp'].fillna(mode_hp2)

In [25]:
df['hp'].isnull().sum()

0

### Column 19: Displacement

In [26]:
mode_displacement = df.groupby(['make_model','Body','hp'])['Displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [27]:
df['Displacement'] = df['Displacement'].fillna(mode_displacement)

In [28]:
mode_displacement1 = df.groupby(['make_model','Body'])['Displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [29]:
df['Displacement'] = df['Displacement'].fillna(mode_displacement1)

In [30]:
mode_displacement2 = df.groupby(['make_model'])['Displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [31]:
df['Displacement'] = df['Displacement'].fillna(mode_displacement2)

In [32]:
df['Displacement'].isnull().sum()

0

### Column: 4: km

In [33]:
df["km"].fillna(df.groupby(['registration','Displacement']).km.transform(lambda x: x.mean()), inplace=True)

In [34]:
mean_group_km1=df.groupby(['registration']).km.transform(lambda x: x.mean())

In [35]:
df["km"].fillna(mean_group_km1, inplace=True)

In [36]:
df["km"].isnull().sum()

0

### Column 6: prev_owner

In [37]:
#If 'km' is less than 10, we can assume that it is a new car
index=df[df["km"]<10].index

In [38]:
df.loc[index,"prev_owner"]=0.0

In [39]:
#If a car's Type is 'New', we can assume that it has only 1 previous owner 
df.loc[(df["Type"]=="New") & (df["prev_owner"].isnull()),"prev_owner"]=1.0

In [40]:
#We can assume that if registration date is 2019 or km is less than 5000, it has only 1 previous owner
df.loc[((df["registration"]==2019) | (df["km"]<5000)) & (df["prev_owner"].isnull()),"prev_owner"]=1.0

In [41]:
#Fill the rest according to registration date mode
mode_group_po = df.groupby(['registration'])['prev_owner'].transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [42]:
df["prev_owner"].fillna(mode_group_po, inplace=True)

In [43]:
df.prev_owner.value_counts(dropna=False)

1.0    14274
0.0      848
2.0      778
3.0       17
4.0        2
Name: prev_owner, dtype: int64

### Column 9: Next Inspection

In [44]:
#Rate of NULL values is too high. We can drop this column too.
df.drop(["Next Inspection"],axis=1,inplace=True)

### Column 10: Inspection new

In [ ]:
#Fill Null values as 'No'
df['Inspection new'] = df['Inspection new'].fillna('No')

### Column 11: Warranty(months)

In [45]:
df['Warranty(months)'].value_counts(dropna=False)

NaN     11066
12.0     2594
24.0     1118
60.0      401
36.0      279
48.0      149
6.0       125
72.0       59
3.0        33
23.0       11
18.0       10
20.0        7
25.0        6
2.0         5
50.0        4
26.0        4
16.0        4
19.0        3
1.0         3
4.0         3
13.0        3
34.0        3
45.0        2
14.0        2
17.0        2
11.0        2
46.0        2
28.0        2
21.0        2
22.0        2
9.0         2
30.0        1
33.0        1
56.0        1
40.0        1
7.0         1
15.0        1
8.0         1
10.0        1
49.0        1
47.0        1
65.0        1
Name: Warranty(months), dtype: int64

In [46]:
#We can assume that if the car has a warranty, seller is more likely to input this info
df['Warranty(months)'] = df['Warranty(months)'].fillna(0.0)

### Column 12: Body Color

In [156]:
#Fill the Null values according to make_model and Body columns
mode_color = df.groupby(['make_model','Body'])['Body Color']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [157]:
df['Body Color'] = df['Body Color'].fillna(mode_color)

In [158]:
df['Body Color'].value_counts(dropna=False)

Black     3941
Grey      3836
White     3469
Silver    1647
Blue      1431
Red        957
Brown      289
Green      158
Beige      108
Yellow      51
Violet      18
Bronze       6
NaN          3
Orange       3
Gold         2
Name: Body Color, dtype: int64

In [159]:
mode_color1 = df.groupby(['make_model'])['Body Color']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [160]:
df['Body Color'] = df['Body Color'].fillna(mode_color1)

In [161]:
df['Body Color'].value_counts(dropna=False)

Black     3941
Grey      3839
White     3469
Silver    1647
Blue      1431
Red        957
Brown      289
Green      158
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: Body Color, dtype: int64

### Column 13: Paint Type

In [149]:
mode_paint = df.groupby(['make_model','Body'])['Paint Type']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [150]:
df['Paint Type'] = df['Paint Type'].fillna(mode_paint)

In [151]:
mode_paint1 = df.groupby(['make_model'])['Paint Type']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [152]:
df['Paint Type'] = df['Paint Type'].fillna(mode_paint1)

### Column 14: Body Color Original

In [153]:
#I have forgotten to drop this column in the previous section
df.drop(["Body Color Original"],axis=1,inplace=True)

### Column 15: Nr. of Doors

In [48]:
#We can fill NULL values according to model and Body of the car
mode_nr_doors = df.groupby(['make_model','Body'])['Nr. of Doors']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [49]:
df['Nr. of Doors'] = df['Nr. of Doors'].fillna(mode_nr_doors)

In [ ]:
df['Nr. of Doors'].value_counts(dropna=False)

In [50]:
mode_nr_doors1 = df.groupby(['make_model'])['Nr. of Doors']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [51]:
df['Nr. of Doors'] = df['Nr. of Doors'].fillna(mode_nr_doors1)

In [52]:
df['Nr. of Doors'].value_counts(dropna=False)

5.0    11787
4.0     3079
3.0      832
2.0      219
7.0        1
1.0        1
Name: Nr. of Doors, dtype: int64

### Column 16: Nr. of Seats

In [53]:
#We can fill NULL values according to model and Body of the car
mode_nr_seats = df.groupby(['make_model','Body'])['Nr. of Seats']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [54]:
df['Nr. of Seats'] = df['Nr. of Seats'].fillna(mode_nr_seats)

In [55]:
df['Nr. of Seats'].value_counts(dropna=False)

5.0    14302
4.0     1127
7.0      362
2.0      119
NaN        6
6.0        2
3.0        1
Name: Nr. of Seats, dtype: int64

In [56]:
mode_nr_seats1 = df.groupby(['make_model'])['Nr. of Seats']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [57]:
df['Nr. of Seats'] = df['Nr. of Seats'].fillna(mode_nr_seats1)

In [58]:
df['Nr. of Seats'].value_counts(dropna=False)

5.0    14308
4.0     1127
7.0      362
2.0      119
6.0        2
3.0        1
Name: Nr. of Seats, dtype: int64

### Column 21: Weight

In [62]:
#Car model and Motor capacity seem ideal columns to fill weight NULLs
mode_weight = df.groupby(['make_model', 'Displacement'])['Weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [63]:
df['Weight'] = df['Weight'].fillna(mode_weight)

In [65]:
df['Weight'].isnull().sum()

210

In [66]:
mode_weight1 = df.groupby(['make_model'])['Weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [67]:
df['Weight'] = df['Weight'].fillna(mode_weight1)

In [68]:
df['Weight'].isnull().sum()

35

In [69]:
mode_weight2 = df.groupby(['make_model', 'hp'])['Weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [70]:
df['Weight'] = df['Weight'].fillna(mode_weight2)

In [71]:
df['Weight'].isnull().sum()

35

In [74]:
mode_weight3 = df.groupby(['Body'])['Weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [75]:
df['Weight'] = df['Weight'].fillna(mode_weight3)

In [76]:
df['Weight'].isnull().sum()

0

### Column 20: Cylinders 

In [77]:
mode_cylinders = df.groupby(['make_model', 'Displacement'])['Cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [60]:
df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders)

In [78]:
df.Cylinders.isnull().sum()

78

In [79]:
mode_cylinders1 = df.groupby(['make_model', 'Weight'])['Cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [80]:
df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders1)

In [81]:
df.Cylinders.isnull().sum()

2

In [82]:
mode_cylinders2 = df.groupby(['make_model'])['Cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [83]:
df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders2)

In [84]:
df.Cylinders.isnull().sum()

1

In [88]:
df[df.Cylinders.isnull()][['make_model', 'price', 'Body', 'Displacement']]

,make_model,price,Body,Displacement
2614,Audi A2,28200.0,Off-Road,1598.0


In [89]:
#We can assume that an Off-Road vehicle have 4 cylinders
df['Cylinders'] = df['Cylinders'].fillna(4.0)

### Column 22: Drive chain

In [90]:
df['Drive chain'].value_counts(dropna=False)

front    8886
NaN      6858
4WD       171
rear        4
Name: Drive chain, dtype: int64

In [91]:
mode_drive_chain = df.groupby(['make_model', 'Body'])['Drive chain']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [92]:
df['Drive chain'] = df['Drive chain'].fillna(mode_drive_chain)

In [93]:
#We still have some NULL values
df['Drive chain'].value_counts(dropna=False)

front    15699
4WD        204
NaN         12
rear         4
Name: Drive chain, dtype: int64

In [94]:
mode_drive_chain1 = df.groupby(['make_model'])['Drive chain']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [95]:
df['Drive chain'] = df['Drive chain'].fillna(mode_drive_chain1)

In [96]:
df['Drive chain'].value_counts(dropna=False)

front    15711
4WD        204
rear         4
Name: Drive chain, dtype: int64

### Column 24: CO2 Emission

In [98]:
mode_CO2 = df.groupby(['make_model','Displacement'])['CO2 Emission']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [99]:
df['CO2 Emission'] = df['CO2 Emission'].fillna(mode_CO2)

In [100]:
mode_CO2_1 = df.groupby(['make_model'])['CO2 Emission']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [101]:
df['CO2 Emission'] = df['CO2 Emission'].fillna(mode_CO2_1)

In [103]:
mode_CO2_2 = df.groupby(['Body'])['CO2 Emission']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [104]:
df['CO2 Emission'] = df['CO2 Emission'].fillna(mode_CO2_2)

In [105]:
df['CO2 Emission'].isnull().sum()

0

### Column 23: Emission Class

In [107]:
mode_Emision_C= df.groupby(['CO2 Emission'])['Emission Class']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [109]:
df['Emission Class'] = df['Emission Class'].fillna(mode_Emision_C)

In [110]:
mode_Emision_C1= df.groupby(['registration'])['Emission Class']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [111]:
df['Emission Class'] = df['Emission Class'].fillna(mode_Emision_C1)

In [112]:
df['Emission Class'].isnull().sum()

0

### Column 24: Gears

In [114]:
mode_Gears= df.groupby(['make_model','Gearing Type'])['Gears']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [115]:
df['Gears'] = df['Gears'].fillna(mode_Gears)

In [116]:
df.Gears.isnull().sum()

9

In [117]:
mode_Gears1= df.groupby(['Gearing Type'])['Gears']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [118]:
df['Gears'] = df['Gears'].fillna(mode_Gears1)

In [119]:
df.Gears.isnull().sum()

0

### Column 25: Country Version

In [120]:
df.drop(['Country version'], axis=1, inplace=True)

### Column 27: Upholstery_Color

In [123]:
df.drop(['Upholstery_Color'], axis=1, inplace=True)

### Column 30: Consumption_combined

In [124]:
coms = ((df['Consumption_city']+df['Consumption_country'])/2)

In [125]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(coms)

In [127]:
mode_Cons= df.groupby(['CO2 Emission'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [128]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons)

In [130]:
mode_Cons1= df.groupby(['Displacement'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [131]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons1)

In [132]:
mode_Cons2= df.groupby(['make_model'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [133]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons2)

In [134]:
df['Consumption_combined'].isnull().sum()

0

In [146]:
df.drop(columns = ['Consumption_city', 'Consumption_country'], inplace = True)

### Column 31: Extras

In [136]:
df['Extras'] = df['Extras'].fillna(0)

### Column 32: Safety & Security

In [137]:
df['Safety & Security'] = df['Safety & Security'].fillna(0)

### Column 33: Entertainment & Media

In [138]:
df['Entertainment & Media'] = df['Entertainment & Media'].fillna(0)

### Column 34: Comfort & Convenience

In [140]:
df['Comfort & Convenience'] = df['Comfort & Convenience'].fillna(0)

## Dataset After Filling NULL Values

In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   make_model             15919 non-null  object 
 1   price                  15919 non-null  float64
 2   vat                    15919 non-null  object 
 3   km                     15919 non-null  float64
 4   registration           15919 non-null  float64
 5   prev_owner             15919 non-null  float64
 6   hp                     15919 non-null  float64
 7   Type                   15919 non-null  object 
 8   Inspection new         15919 non-null  object 
 9   Warranty(months)       15919 non-null  float64
 10  Body Color             15919 non-null  object 
 11  Paint Type             15919 non-null  object 
 12  Body                   15919 non-null  object 
 13  Nr. of Doors           15919 non-null  float64
 14  Nr. of Seats           15919 non-null  float64
 15  Ge

In [163]:
#Save the dataframe as a csv file
df.to_csv('auto_scout_filled.csv')